In [1]:
%matplotlib inline 

import sys 
import xarray as xr
import numpy as np
import os 
import math

coords = xr.open_dataset('/home/jvalenti/MOAD/grid/coordinates_seagrid_SalishSea201702.nc', decode_times=False)
mask = xr.open_dataset('/home/jvalenti/MOAD/grid2/mesh_mask202108_TDV.nc')

In [4]:
def count_inside_grid_cell(center_x, center_y, cell_width, cell_height, x,y):
    cell_width = cell_width/(deg2met*latT(center_y))
    cell_height = cell_height/deg2met
    min_x = np.array(center_x - cell_width / 2)
    max_x = np.array(center_x + cell_width / 2)
    min_y = np.array(center_y - cell_height / 2)
    max_y = np.array(center_y + cell_height / 2)
    inside_mask = np.logical_and.reduce([
        min_x[:, np.newaxis] <= x,
        x <= max_x[:, np.newaxis],
        min_y[:, np.newaxis] <= y,
        y <= max_y[:, np.newaxis]
    ])
    c = np.sum(inside_mask,axis=1)
    return c
ds = xr.open_dataset('/home/jvalenti/MOAD/outfall_30days.nc')
MFc = 1e7
zlevels = [0,10,50,150,800]
DS=ds.to_dataframe()
DS = DS[DS.z < 800]
DS = DS[DS.status==1]
lat = coords.nav_lat
lon = coords.nav_lon
cell_width = coords.e1t[0,:,:]
cell_height = coords.e2t[0,:,:]
deg2met = 111319.5
x = np.array(DS.lon)
y = np.array(DS.lat)
z = np.array(DS.z)


cellvol = np.array(1e7/DS.cellvol)
def latT(lat):
    return np.cos(lat*(math.pi/180))

#result = count_inside_grid_cell(lon[0,0], lat[0,0], cell_width[0,0], cell_height[0,0],x,y)
conc2 = np.zeros((len(zlevels),coords.nav_lon.shape[0],coords.nav_lon.shape[1]))
for k in range(len(zlevels)-2,-1,-1):
    print(f'{k} index starting.')  
    for j in range(5):
        zmin = int(zlevels[k])
        zmax = int(zlevels[k+1])
        X = x[np.logical_and(z >= zmin, z < zmax)]
        Y = y[np.logical_and(z >= zmin, z < zmax)]
        conc2[k,j,:]+= count_inside_grid_cell(lon[j,:], lat[j,:], cell_width[j,:], cell_height[j,:],X,Y)*MFc/(cell_width[j,:]* cell_height[j,:]*(zmax-zmin))

3 index starting.
2 index starting.
1 index starting.
0 index starting.


In [4]:
conc = np.load('/ocean/jvalenti/MOAD/concentration_30days.npy')

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

    